# KLOOK_주문처리_자동화
### 프로세스 :
- 1. Gmail API 연동 후 주문 메일 가져오기
- 2. 가져온 주문 정보를 KLOOK 주문 사이트와 연동
- 3. 연동된 데이터를 전처리 및 암호화 후 API주소에 송신
- 4. 송신된 데이터는 DB에서 확인 가능
- 5. DB에 들어간 데이터를 확인 후 생성한 바우처 번호를 다시 KLOOK 측에 전송 및 등록


## 라이브러리 설치, 환경 설정

In [4]:
import datetime as dt, pandas as pd, numpy as np, datetime as dt
import os, time , glob, shutil, pymysql, re, requests, pickle, os.path, base64, email, hashlib, crypto, random
from openpyxl import Workbook, load_workbook
from pathlib import Path
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from googleapiclient.errors import HttpError
#from bs4 import BeautifulSoup
from __future__ import print_function
from base64 import b64encode
sys.modules['Crypto'] = crypto
from Crypto.Util.Padding import pad
from Crypto.Cipher import AES


def live_db_conn() :
    conn = pymysql.connect(host='링크', user='사용자명', password='비밀번호',autocommit=True,cursorclass=pymysql.cursors.DictCursor, db = "db명")
    return conn
def get_path(path) : 
    return driver.find_elements(by = By.CSS_SELECTOR, value = path)
def get_class(path) : 
    return driver.find_elements(by = By.CLASS_NAME, value = path)
def get_replace(string) : 
    return string.replace("\t",'')
def get_Xpath(path) : 
    return driver.find_elements(by = By.XPATH, value = path)


def data_encoder(text):
    if len(text)>0:
        message = email.message_from_string(str(base64.urlsafe_b64decode(text), 'utf-8'))
    return message

# reserve_time 전처리
def change_time(x) :
    if 'pm' in x :
        if len(re.sub(r'[^0-9]','',x)) == 3 :
            x = '{}:{}'.format(12+int(x[0]),x[1:]).replace("::",':')
        elif len(x.replace("pm",'')) == 1 :
            x = "{}:00".format(int(x.replace("pm",'')) + 12)
    elif len(x)== len(re.sub(r'[^0-9]','',x)) == 4 :
        x = "{}:{}".format(x[:2],x[2:]) 
    elif len(re.sub(r'[^0-9]','',x)) == 3 :
        x = "{}{}".format(int(x[0])+12, x[1:]) 
    else : x
    return x.replace("pm",'').replace("::",':')

# booking time
def get_date(string) :
    y, m = string.strip()[12:].split()[0], dt.datetime.strptime(string.strip()[8:].split()[0],'%b').month#.tm_mon
    d,t = string.strip()[5:].split()[0]  , string.strip()[17:25]
    bt = "{}:{}:{} {}".format(y,m,d,t)
    result = dt.datetime.strptime(bt , "%Y:%m:%d %H:%M:%S") + dt.timedelta(hours=16)
    return result.strftime("%Y-%m-%d %H:%M:%S")

def encryptString(str, secret_key, secret_iv):
    key = hashlib.sha256(secret_key.encode('utf-8')).hexdigest()[:32].encode('utf-8')
    iv = hashlib.sha256(secret_iv.encode('utf-8')).hexdigest()[:16].encode('utf-8')
    data = pad(str.encode(), AES.block_size)
    cipher = AES.new(key, AES.MODE_CBC, iv)
    return b64encode(b64encode(cipher.encrypt(data)).decode("utf-8").encode()).decode("utf-8").replace("=","")

URL_LIVE = 'API 주소'

creds, SCOPES = None, ['https://www.googleapis.com/auth/gmail.readonly']
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)


## Driver 세팅 및 KLOOK 로그인

In [2]:
# 드라이버 세팅
driver = webdriver.Chrome(r"{}\chromedriver.exe".format(os.getcwd()))
driver.get('https://merchant.klook.com/ko/booking')

# URL
url = 'https://merchant.klook.com/ko/home'
now = time

# 아이디 비밀번호
site_id,site_pw = "KLOOK 관리자 ID", "KLOOK 관리자 비밀번호"

time.sleep(random.uniform(2,5))
# 로그인

get_class('ant-tabs-tab')[2].click()
time.sleep(random.uniform(1,2))
get_Xpath('//*[@id="app"]/section/main/div/div[2]/div/div/div[1]/form/div[1]/div[3]/div[3]/div[2]/div/div/span/span/input')[0].clear()
get_Xpath('//*[@id="app"]/section/main/div/div[2]/div/div/div[1]/form/div[1]/div[3]/div[3]/div[2]/div/div/span/span/input')[0].send_keys(site_id)
time.sleep(random.uniform(1,2))
get_Xpath('//*[@id="app"]/section/main/div/div[2]/div/div/div[1]/form/div[2]/div/div/span/span/input')[0].clear()
get_Xpath('//*[@id="app"]/section/main/div/div[2]/div/div/div[1]/form/div[2]/div/div/span/span/input')[0].send_keys(site_pw)
time.sleep(random.uniform(2,3))
get_Xpath('//*[@id="app"]/section/main/div/div[2]/div/div/div[1]/form/div[4]/div/div/span/button')[0].click()
time.sleep(random.uniform(2,3))

<ipython-input-2-61e050a2ee5e>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"{}\chromedriver.exe".format(os.getcwd()))


## 주문 정보 전처리 및 API 전송 후 바우처 번호 등록

In [17]:
#def work(w_time) :
#for t in range(w_time) :
creds, SCOPES = None, ['https://www.googleapis.com/auth/gmail.readonly']
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

#for i in range(w_time) :
driver.refresh()

conn  = live_db_conn()
curs  = conn.cursor()
start = time.time()

sql = """
SELECT OS.store_id, S.name ,OS.store_id_ext, OS.memo, SC.order_confirm
FROM ota_store OS, store S ,store_contract SC
WHERE OS.ota_id = 3 AND OS.store_id = S.id AND SC.store_id = S.id
"""
curs.execute(sql)
otas = pd.DataFrame(curs.fetchall())

sql = """
SELECT OP.memo, OP.product_id, OP.product_id_ext, price
FROM ota_product OP
WHERE OP.ota_id = 3
"""
curs.execute(sql)
otap = pd.DataFrame(curs.fetchall())

sql = """
SELECT O.store_id, case when O.confirm_at is null then 0 else 1 end 'confirm_at', O.ota_order_no , O.manager_memo, O.name, O.status
FROM `order` O
WHERE O.ota_id = 3
"""
curs.execute(sql)
order = pd.DataFrame(curs.fetchall())

print('db 가져오기 완료')#, "\n쿼리 작동 시간 :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간 

service = build('gmail', 'v1', credentials=creds)
# 취소분 반영
cancel_list = service.users().messages().list(userId='me', q='subject:Klook Order Canceled').execute()
messages = cancel_list['messages']
cl = []
for message_id in messages: 
    message = service.users().messages().get(userId='me', id= message_id['id']).execute()
    if 'CONFIRMED order has now been cancelled' in message['snippet'] and pd.read_html(str(data_encoder(message['payload']['body']['data'])))[6][0][0].split()[-1] in set(order[order['status'] != 'cancel']['ota_order_no'].dropna()):
        print(pd.read_html(str(data_encoder(message['payload']['body']['data'])))[6][0][0].split()[-1])
        cl.append(pd.read_html(str(data_encoder(message['payload']['body']['data'])))[6][0][0].split()[-1])
        
if len(cl) > 0 :
    sql = """
    UPDATE `order` O
    SET O.status = 'cancel'
    WHERE O.ota_order_no IN {}
    AND O.status != 'cancel'
    """.format(tuple(cl))
    curs.execute(sql)
    print('취소분 반영완료')
curs.close()
conn.close()

results = service.users().messages().list(userId='me', q='subject:Klook has received an order').execute()
messages = results['messages']

# 제거할 단어
re_word = "Booking reference ID:|Date Request:|Time Request:|Lead participant:|Country/region of passport:|Participant:|Activity URL:|Number of Diners:"#|x Set|x Person" 
# 기본 데이터프레임 틀
df = pd.DataFrame(columns= ['store_id', 'products', 'total_price', 'pay_method', 'currency','booking_time', 'reserve_day', 'reserve_time', 'name', 'email', 'phone','memo', 'person', 'ota_code', 'ota_key', 'ota_sns_type', 'ota_sns_id','ota_country', 'ota_order_no', 'confirm_at', 'order_confirm'])

omm = order[order['manager_memo'].apply(lambda x : x[-20:-14] if type(x) == str else '') == dt.datetime.now().strftime("%y%m%d")]        
if len(omm) > 0 :    
    voucher_num = max(omm['manager_memo'].apply(lambda x : int(x[-4:])))
else :
    voucher_num = 0

for message_id in messages:
    message = service.users().messages().get(userId='me', id= message_id['id']).execute()
    contents = pd.read_html(str(data_encoder(message['payload']['body']['data'])))[6][0]
    sn, sie = message['snippet'][51:].split("-")[0].strip() , contents[6].split("/")[-1]
    pn = message['snippet'][51:].split(" - ")[1].split("Please")[0][:-2].replace("amp;",'')
    
    tmp = pd.DataFrame(map(lambda x : re.sub(re_word ,"", x).strip(), contents[:6].values)).transpose()
    tmp.columns = ['ota_order_no','reserve_day','reserve_time','name','ota_country','person']
    tmp['memo'] = 'X'
    
    for content in contents[8:] :
        if 'Dining Time' in content :
            if content.split()[-1] in ('00','0','am','pm') :
                tmp['reserve_time'] = ''.join(content.split()[-2:])
            else :
                tmp['reserve_time'] = content.split()[-1]
            #re.sub(r'\([^)]*\)|', '', ed)
        elif 'Number of Diners:' in content :
            tmp['person'] = str(re.sub(re_word,'', content)).strip()
        else :
            tmp['memo'] = content
    tmp['booking_time'] = get_date(message['payload']['headers'][3]['value'].split(";")[1])[:-3]
    #if sn in dict(otas[['memo','store_id']].values) or :
    #    tmp['store_id'] = dict(otas[['memo','store_id']].values).get(sn)
    if sie in dict(otas[['store_id_ext','store_id']].values) :
        tmp['store_id'] = dict(otas[['store_id_ext','store_id']].values).get(sie)
        tmp['order_confirm'] = dict(otas[['store_id_ext','order_confirm']].values).get(sie)
        tmp['confirm_at'] = dict(order[['store_id','confirm_at']].values).get(tmp['store_id'][0])
    if pn in dict(otap[['memo','product_id']].values):
        tmp['products'] = [{"product_id" : str(dict(otap[['memo','product_id']].values).get(pn)) , 'product_price' : int(dict(otap[['memo','price']].values).get(pn)), 'product_quantity' : int(re.sub(r'[^0-9]','',contents[5]))}]
        voucher_num += 1
        if  len(str(voucher_num)) == 1 :
            tmp['memo'] = "바우처 번호 : {}-REDTABLE-000{} & 요청사항 : {}".format(dt.datetime.now().strftime("%y%m%d"), voucher_num, tmp['memo'][0])
        else :
            tmp['memo'] = "바우처 번호 : {}-REDTABLE-00{} & 요청사항 : {}".format(dt.datetime.now().strftime("%y%m%d"), voucher_num, tmp['memo'][0])
    else :
        print('db에 프로덕트 없음', pn)
    
    df = pd.concat([df,tmp])
dd = df.copy()
df['reserve_time'] = df['reserve_time'].apply(lambda x : change_time(x))
df['person'] = df['person'].apply(lambda x : sum(map(lambda num : int(num), re.findall(r'\d+' ,str(x)))))
df['name'] = df['name'].apply(lambda x : x.replace("()",'').replace(")",') '))
df['total_price'] = df['products'].apply(lambda x : str(x['product_quantity'] * x['product_price']) if type(x)!= float else '')
df['products'] = df['products'].apply(lambda x : str([x]) if type(x) == dict else x)
df[['pay_method','currency','ota_code','ota_key']] = ['card','KRW', 'klook', 'L2w5SGIvUFcrbzZzc3huSnVZSjhnZz09']
df[['store_id','total_price','person']] = df[['store_id','total_price','person']].astype(str)
df['memo'] = df['memo'].replace("\xa0",' ')
df.fillna(inplace = True, value='')

secret_key, secret_iv = "redtabletoken", "secret iv"
for i in range(len(df)) :
    if df.iloc[i]['ota_order_no'] in set(order['ota_order_no'].dropna()) :
        continue
    else :
        print('{} : 등록필요'. format(df.iloc[i]['ota_order_no']))
    PARAMS = str(df.iloc[i,:-2].to_dict())
    data = PARAMS.replace("'",'"').replace(" && ","&&").replace('"[','[').replace(']"',']')
    r = encryptString(data, secret_key, secret_iv).replace("=",'')
    print(data, r)
    req = requests.post(url = URL_LIVE, data = {'data' : r})
    if req.status_code == 200 :
        print('{} : DB 등록완료'.format(df.iloc[i]['ota_order_no']))
    else : 
        print('{} : 전송실패 {}'.format(df.iloc[i]['ota_order_no'] , req.content))
        
k_list = list(map(lambda x: x.split("\n")[0] ,get_class("booking-list")[0].text[:-8].split("예약 확인 ID: ")[1:]))
for i,j in df[df['order_confirm']==0][['ota_order_no','memo']].values :
        if i in k_list :
            try :
                print("{} : 바우처 등록필요, 바우처 번호 : {}".format(i,j))
                get_class("ant-input")[1].clear()
                print("예약확인 ID 입력")
                get_class("ant-input")[1].send_keys(i)
                get_class("ant-input")[1].send_keys(Keys.RETURN)
                time.sleep(random.uniform(4,5))
                print('확정버튼 클릭전')
                get_class("ant-btn")[list(map(lambda x : x.text, get_class("ant-btn"))).index("확정")].click()
                print('확정버튼 클릭후')
                print('바우처 번호 입력전')
                time.sleep(random.uniform(1,2))
                get_class("ant-input")[get_class("ant-modal-body")[0].text.split("\n").index("바우처 번호")+3].clear()
                time.sleep(random.uniform(1,2))
                get_class("ant-input")[get_class("ant-modal-body")[0].text.split("\n").index("바우처 번호")+3].send_keys(j[9:29])
                print('바우처 번호 입력후, 확정 클릭 전')
                try :
                    get_class("ant-btn")[-1].click()
                except :
                    get_path("body > div:nth-child(3) > div > div.ant-modal-wrap > div > div.ant-modal-content > div.ant-modal-footer > div > button.ant-btn.ant-btn-primary")[0].click()
                driver.refresh()
                time.sleep(random.uniform(3,5))
                print("{} : 바우처 KLOOK에 등록 완료".format(j[9:29]))
            except :
                print('바우처 등록실패 -> 코드 수정 필요')
        #else :
        #    print("{}: 이미 확정상태 , 바우처 번호 : {}".format(i, j[9:29]))
print("과정 완료, 작동 시간 :", time.time() - start)

#if w_time != 1 :
#    time.sleep(1800)

db 가져오기 완료
과정 완료, 작동 시간 : 15.997133731842041
